In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ast
import pickle
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import keras
from keras.models import Sequential
from keras.layers import Input, Embedding, Conv1D, MaxPool1D, GlobalMaxPooling1D, Dense, Dropout
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam
import keras_tuner as kt
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
EPOCHS=500
BATCH_SIZE=128
PATIENCE=5

In [4]:
# base_path = '/content/drive/MyDrive/fake_news/'
base_path = ''

# Load Embeddings & Inputs

In [5]:
with open(base_path+'artifacts/pretrained_embeddings_inputs.pkl', 'rb') as f:
    loaded_input_items = pickle.load(f)

In [6]:
embedding_matrix = loaded_input_items['embedding_matrix']
X_train_pad = loaded_input_items['X_train_pad']
X_val_pad = loaded_input_items['X_val_pad']
X_test_pad = loaded_input_items['X_test_pad']
y_train = loaded_input_items['y_train']
y_val = loaded_input_items['y_val']
y_test = loaded_input_items['y_test']

In [7]:
VOCAB_SIZE = embedding_matrix.shape[0]
EMBEDDING_DIM = embedding_matrix.shape[1]
MAX_LEN = len(X_train_pad[0])

In [8]:
print(f"Embedding dimension: {EMBEDDING_DIM}\nVocab size: {VOCAB_SIZE}\nMaximum input length: {MAX_LEN}")

Embedding dimension: 100
Vocab size: 35756
Maximum input length: 588


# Basic CNN

In [9]:
model = Sequential()
model.add(Input(shape=(MAX_LEN,)))
model.add(Embedding(input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM,
                        weights=[embedding_matrix],
                        trainable=True))
model.add(Conv1D(filters=128, kernel_size=5, strides=1, 
                     padding='valid', activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=PATIENCE,
                      restore_best_weights=True, verbose=1)
model.fit(X_train_pad, y_train,
             validation_data=(X_val_pad, y_val),
             epochs=EPOCHS, batch_size=BATCH_SIZE,
             callbacks=[estop], verbose=1)

Epoch 1/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 12s 96ms/step - accuracy: 0.7593 - loss: 0.5093 - val_accuracy: 0.8902 - val_loss: 0.2771
Epoch 2/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 10s 98ms/step - accuracy: 0.9232 - loss: 0.2020 - val_accuracy: 0.9273 - val_loss: 0.1806
Epoch 3/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 10s 98ms/step - accuracy: 0.9673 - loss: 0.1035 - val_accuracy: 0.9440 - val_loss: 0.1388
Epoch 4/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 10s 98ms/step - accuracy: 0.9896 - loss: 0.0462 - val_accuracy: 0.9474 - val_loss: 0.1321
Epoch 5/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 10s 98ms/step - accuracy: 0.9975 - loss: 0.0199 - val_accuracy: 0.9474 - val_loss: 0.1393
Epoch 6/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 10s 98ms/step - accuracy: 0.9990 - loss: 0.0100 - val_accuracy: 0.9507 - val_loss: 0.1431
Epoch 7/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 10s 99ms/step - accuracy: 0.9998 - loss: 0.0054 - val_accuracy: 0.9492 - val_loss: 0.1503
Epoch 8/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 10s 98ms/step - accuracy: 1.0000 - loss: 0

In [11]:
test_loss, test_accuracy = model.evaluate(X_test_pad, y_test, verbose=0)
print("Basic CNN Test Accuracy:", test_accuracy)
print("Basic CNN Test Loss:", test_loss)

Basic CNN Test Accuracy: 0.9532846808433533
Basic CNN Test Loss: 0.1253475695848465


# CNN with previously tuned Hyperparameters

{'embeddings_trainable': 1, 'conv1_filters': 512, 'conv1_kernel_size': 4, 'dense_units': 16, 'dense_dropout': 0.30000000000000004, 'learning_rate': 0.001}

In [12]:
model = Sequential()
model.add(Input(shape=(MAX_LEN,)))
model.add(Embedding(input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM,
                        weights=[embedding_matrix],
                        trainable=True))
model.add(Conv1D(filters=512, kernel_size=4,
                     strides=1, padding='valid', activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(units=16, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=Adam(learning_rate=0.001), 
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [13]:
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=PATIENCE,
                      restore_best_weights=True, verbose=1)
model.fit(X_train_pad, y_train,
             validation_data=(X_val_pad, y_val),
             epochs=EPOCHS, batch_size=BATCH_SIZE,
             callbacks=[estop], verbose=1)

Epoch 1/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 31s 288ms/step - accuracy: 0.7550 - loss: 0.4677 - val_accuracy: 0.9118 - val_loss: 0.2449
Epoch 2/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 29s 278ms/step - accuracy: 0.9216 - loss: 0.2399 - val_accuracy: 0.9431 - val_loss: 0.1642
Epoch 3/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 29s 278ms/step - accuracy: 0.9633 - loss: 0.1635 - val_accuracy: 0.9529 - val_loss: 0.1392
Epoch 4/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 29s 283ms/step - accuracy: 0.9822 - loss: 0.1227 - val_accuracy: 0.9556 - val_loss: 0.1251
Epoch 5/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 29s 279ms/step - accuracy: 0.9922 - loss: 0.0983 - val_accuracy: 0.9529 - val_loss: 0.1506
Epoch 6/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 29s 278ms/step - accuracy: 0.9958 - loss: 0.0869 - val_accuracy: 0.9592 - val_loss: 0.1328
Epoch 7/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 29s 277ms/step - accuracy: 0.9983 - loss: 0.0758 - val_accuracy: 0.9608 - val_loss: 0.1343
Epoch 8/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 29s 278ms/step - accuracy: 0.9990 -

In [14]:
test_loss, test_accuracy = model.evaluate(X_test_pad, y_test, verbose=0)
print("Tuned CNN Test Accuracy:", test_accuracy)
print("Tuned CNN Test Loss:", test_loss)

Tuned CNN Test Accuracy: 0.9579075574874878
Tuned CNN Test Loss: 0.12192881852388382


# Increasing Patience & Adding Learning Rate Schedule

In [16]:
model2 = Sequential()
model2.add(Input(shape=(MAX_LEN,)))
model2.add(Embedding(input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM,
                        weights=[embedding_matrix],
                        trainable=True))
model2.add(Conv1D(filters=512, kernel_size=4,
                     strides=1, padding='valid', activation='relu'))
model2.add(GlobalMaxPooling1D())
model2.add(Dense(units=16, activation='relu'))
model2.add(Dropout(0.3))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(optimizer=Adam(learning_rate=0.001), 
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [17]:
PATIENCE_ES = 10
PATIENCE_LR = 5
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=PATIENCE_ES,
                      restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=PATIENCE_LR, min_delta=1e-4,
                              min_lr=1e-5, verbose=1)
model2.fit(X_train_pad, y_train,
             validation_data=(X_val_pad, y_val),
             epochs=EPOCHS, batch_size=BATCH_SIZE,
             callbacks=[estop, reduce_lr], verbose=1)

Epoch 1/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 33s 310ms/step - accuracy: 0.8247 - loss: 0.3948 - val_accuracy: 0.9151 - val_loss: 0.2099 - learning_rate: 0.0010
Epoch 2/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 31s 302ms/step - accuracy: 0.9448 - loss: 0.1545 - val_accuracy: 0.9462 - val_loss: 0.1407 - learning_rate: 0.0010
Epoch 3/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 29s 279ms/step - accuracy: 0.9754 - loss: 0.0742 - val_accuracy: 0.9523 - val_loss: 0.1209 - learning_rate: 0.0010
Epoch 4/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 29s 286ms/step - accuracy: 0.9916 - loss: 0.0338 - val_accuracy: 0.9544 - val_loss: 0.1212 - learning_rate: 0.0010
Epoch 5/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 30s 288ms/step - accuracy: 0.9967 - loss: 0.0191 - val_accuracy: 0.9565 - val_loss: 0.1261 - learning_rate: 0.0010
Epoch 6/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 29s 285ms/step - accuracy: 0.9982 - loss: 0.0118 - val_accuracy: 0.9574 - val_loss: 0.1350 - learning_rate: 0.0010
Epoch 7/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 29s 277ms/step - accura

In [18]:
test_loss2, test_accuracy2 = model2.evaluate(X_test_pad, y_test, verbose=0)
print("Tuned Scheduled CNN Test Accuracy:", test_accuracy2)
print("Tuned Scheduled CNN Test Loss:", test_loss2)

Tuned Scheduled CNN Test Accuracy: 0.9569343328475952
Tuned Scheduled CNN Test Loss: 0.11684489995241165


# Increasing Patience

In [23]:
model3 = Sequential()
model3.add(Input(shape=(MAX_LEN,)))
model3.add(Embedding(input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM,
                        weights=[embedding_matrix],
                        trainable=True))
model3.add(Conv1D(filters=512, kernel_size=4,
                     strides=1, padding='valid', activation='relu'))
model3.add(GlobalMaxPooling1D())
model3.add(Dense(units=16, activation='relu'))
model3.add(Dropout(0.3))
model3.add(Dense(1, activation='sigmoid'))
model3.compile(optimizer=Adam(learning_rate=0.001), 
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [24]:
PATIENCE_ES = 10
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=PATIENCE_ES,
                      restore_best_weights=True, verbose=1)
model3.fit(X_train_pad, y_train,
             validation_data=(X_val_pad, y_val),
             epochs=EPOCHS, batch_size=BATCH_SIZE,
             callbacks=[estop], verbose=1)

Epoch 1/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 33s 306ms/step - accuracy: 0.7963 - loss: 0.4370 - val_accuracy: 0.9158 - val_loss: 0.2330
Epoch 2/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 30s 289ms/step - accuracy: 0.9326 - loss: 0.1859 - val_accuracy: 0.9468 - val_loss: 0.1505
Epoch 3/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 29s 280ms/step - accuracy: 0.9676 - loss: 0.1058 - val_accuracy: 0.9425 - val_loss: 0.1501
Epoch 4/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 31s 302ms/step - accuracy: 0.9849 - loss: 0.0582 - val_accuracy: 0.9565 - val_loss: 0.1197
Epoch 5/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 31s 302ms/step - accuracy: 0.9935 - loss: 0.0325 - val_accuracy: 0.9586 - val_loss: 0.1229
Epoch 6/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 32s 311ms/step - accuracy: 0.9966 - loss: 0.0183 - val_accuracy: 0.9577 - val_loss: 0.1299
Epoch 7/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 32s 307ms/step - accuracy: 0.9983 - loss: 0.0112 - val_accuracy: 0.9586 - val_loss: 0.1438
Epoch 8/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 30s 291ms/step - accuracy: 0.9986 -

In [25]:
test_loss3, test_accuracy3 = model3.evaluate(X_test_pad, y_test, verbose=0)
print("Tuned CNN2 Test Accuracy:", test_accuracy3)
print("Tuned CNN2 Test Loss:", test_loss3)

Tuned CNN2 Test Accuracy: 0.9574209451675415
Tuned CNN2 Test Loss: 0.12015752494335175


# Tuned CNN - 2 Convolution layers

{'embeddings_trainable': 0, 'conv1_filters': 256, 'conv1_kernel_size': 5, 'conv2_filters': 512, 'conv2_kernel_size': 5, 'dense_units': 128, 'dense_dropout': 0.4, 'learning_rate': 0.001}

In [30]:
model_2l = Sequential()
model_2l.add(Input(shape=(MAX_LEN,)))
model_2l.add(Embedding(input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM,
                        weights=[embedding_matrix],
                        trainable=True))

model_2l.add(Conv1D(filters=256, kernel_size=5,
                 strides=1, padding='valid', activation='relu'))
model_2l.add(MaxPool1D(pool_size=2))
model_2l.add(Conv1D(filters=512, kernel_size=5, 
                 strides=1, padding='valid', activation='relu'))
model_2l.add(GlobalMaxPooling1D())
model_2l.add(Dense(units=128, activation='relu'))
model_2l.add(Dropout(0.4))
model_2l.add(Dense(1, activation='sigmoid'))
model_2l.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [31]:
PATIENCE_ES = 10
PATIENCE_LR = 5
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=PATIENCE_ES,
                      restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=PATIENCE_LR, min_delta=1e-4,
                              min_lr=1e-5, verbose=1)
model_2l.fit(X_train_pad, y_train,
             validation_data=(X_val_pad, y_val),
             epochs=EPOCHS, batch_size=BATCH_SIZE,
             callbacks=[estop, reduce_lr], verbose=1)

Epoch 1/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 48s 458ms/step - accuracy: 0.7848 - loss: 0.4502 - val_accuracy: 0.8808 - val_loss: 0.2688 - learning_rate: 0.0010
Epoch 2/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 45s 439ms/step - accuracy: 0.9376 - loss: 0.1681 - val_accuracy: 0.9142 - val_loss: 0.2037 - learning_rate: 0.0010
Epoch 3/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 45s 441ms/step - accuracy: 0.9741 - loss: 0.0736 - val_accuracy: 0.9450 - val_loss: 0.1456 - learning_rate: 0.0010
Epoch 4/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 45s 441ms/step - accuracy: 0.9946 - loss: 0.0223 - val_accuracy: 0.9489 - val_loss: 0.1656 - learning_rate: 0.0010
Epoch 5/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 45s 440ms/step - accuracy: 0.9988 - loss: 0.0060 - val_accuracy: 0.9456 - val_loss: 0.2234 - learning_rate: 0.0010
Epoch 6/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 46s 443ms/step - accuracy: 0.9993 - loss: 0.0037 - val_accuracy: 0.9413 - val_loss: 0.2393 - learning_rate: 0.0010
Epoch 7/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 45s 440ms/step - accura

In [32]:
test_loss_2l, test_accuracy_2l = model_2l.evaluate(X_test_pad, y_test, verbose=0)
print("2 Layer Tuned Scheduled CNN Accuracy:", test_accuracy_2l)
print("2 Layer Tuned Scheduled CNN Loss:", test_loss_2l)

2 Layer Tuned Scheduled CNN Accuracy: 0.9467153549194336
2 Layer Tuned Scheduled CNN Loss: 0.15035033226013184


# Tuned CNN - 3 Convolution layers

{'embeddings_trainable': 0, 'conv1_filters': 128, 'conv1_kernel_size': 5, 'conv2_filters': 128, 'conv2_kernel_size': 3, 'conv3_filters': 512, 'conv3_kernel_size': 4, 'dense_units': 16, 'dense_dropout': 0.30000000000000004, 'learning_rate': 0.001}

In [34]:
model_3l = Sequential()
model_3l.add(Input(shape=(MAX_LEN,)))
model_3l.add(Embedding(input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM,
                        weights=[embedding_matrix],
                        trainable=True))
model_3l.add(Conv1D(filters=128, kernel_size=5,
                 strides=1, padding='valid', activation='relu'))
model_3l.add(MaxPool1D(pool_size=2))
model_3l.add(Conv1D(filters=128, kernel_size=3,
                 strides=1, padding='valid', activation='relu'))
model_3l.add(MaxPool1D(pool_size=2))
model_3l.add(Conv1D(filters=512, kernel_size=4,
                 strides=1, padding='valid', activation='relu'))
model_3l.add(GlobalMaxPooling1D())
model_3l.add(Dense(units=16, activation='relu'))
model_3l.add(Dropout(0.3))
model_3l.add(Dense(1, activation='sigmoid'))
model_3l.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [35]:
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=PATIENCE,
                      restore_best_weights=True, verbose=1)
model_3l.fit(X_train_pad, y_train,
             validation_data=(X_val_pad, y_val),
             epochs=EPOCHS, batch_size=BATCH_SIZE,
             callbacks=[estop], verbose=1)

Epoch 1/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 20s 185ms/step - accuracy: 0.5043 - loss: 0.6962 - val_accuracy: 0.5055 - val_loss: 0.6931
Epoch 2/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 20s 190ms/step - accuracy: 0.5056 - loss: 0.6931 - val_accuracy: 0.5055 - val_loss: 0.6931
Epoch 3/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 19s 184ms/step - accuracy: 0.5056 - loss: 0.6931 - val_accuracy: 0.5055 - val_loss: 0.6931
Epoch 4/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 18s 174ms/step - accuracy: 0.5056 - loss: 0.6931 - val_accuracy: 0.5055 - val_loss: 0.6931
Epoch 5/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 18s 174ms/step - accuracy: 0.5056 - loss: 0.6931 - val_accuracy: 0.5055 - val_loss: 0.6931
Epoch 6/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 18s 176ms/step - accuracy: 0.5056 - loss: 0.6931 - val_accuracy: 0.5055 - val_loss: 0.6931
Epoch 7/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 18s 174ms/step - accuracy: 0.5056 - loss: 0.6931 - val_accuracy: 0.5055 - val_loss: 0.6931
Epoch 8/500
103/103 ━━━━━━━━━━━━━━━━━━━━ 18s 174ms/step - accuracy: 0.5056 -

In [36]:
test_loss_3l, test_accuracy_3l = model_3l.evaluate(X_test_pad, y_test, verbose=0)
print("3 Layer Tuned Scheduled CNN Accuracy:", test_accuracy_3l)
print("3 Layer Tuned Scheduled CNN Loss:", test_loss_3l)

3 Layer Tuned Scheduled CNN Accuracy: 0.5055961012840271
3 Layer Tuned Scheduled CNN Loss: 0.693085253238678


In [37]:
model.save(base_path+'artifacts/pretrained_cnn.keras')